<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2021**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai<br/>
<hr style='height:2px'>

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain questions about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using [Jupyter Notebooks](https://jupyter.org/), which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with [pandas](https://pandas.pydata.org/) as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- <span style='color:purple'>**Web scraping requests can take several minutes**</span>. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, <span style='color:purple'>the order in which you run cells matters</span> and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All"). Just make sure to not re-run the time intensive tasks unnecessarily. In this notebook for example, you could declare a variable to act as a 'setting' and use some controll logic to prevent a re-scrap from happening when not desired.

- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data with [Requests](https://docs.python-requests.org/en/master/) and then manipulating it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). 

But first, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [2]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import pickle # for loading a dictionary from disk
from typing import Optional # typehint that value can also be None

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# data/
# state_data/
state_dir = "state_data/"

In [3]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [4]:
# YOUR CODE HERE
home_page = requests.get(base_url)
# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- **When working with Jupyter Notebooks, avoiding unnecessarily long output in is essential.** Write code that prints the first 10,000 characters from the contents of `home_page` and [enable scolling output for the cell](https://www.youtube.com/watch?v=U4usAUZCv_c&t=1s).</div>


In [5]:
# YOUR CODE HERE
print(home_page.status_code)
print(home_page.text[:10000])
# END OF YOUR CODE HERE

200
<!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"1475026924","applicationTime":2161.554995,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"50a3b7d4afb96077","agentToken":null}; (window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"1475026924"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n){return function(){return o(t,[u.now()].concat(f(arguments)),e?null:this,n),e?void 0:this}}var o=t("handle"),a=t(8),f=t(9),c=

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [6]:
# YOUR CODE HERE
bs_page = BeautifulSoup(home_page.content, "html.parser")
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use [Regular Expressions]('https://docs.python.org/3/library/re.html'), in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate a `state_urls` [dictionary]('https://docs.python.org/3/tutorial/datastructures.html#dictionaries') by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later exercises go smoothly.
- Some HTML tag attributes may change over time. It your code stops working, make sure you are not targeting such ephemeral elements ('jss' class attributes are a common culprit)
</div>

In [7]:
state_urls = {}

# YOUR CODE HERE
for link in bs_page.findAll('a', attrs={'href': re.compile("/visualizations/coronavirus-covid-19-spread-map/state/")}):
    state_urls[link.text] = "https://usafacts.org" + link.get('href')
# END OF YOUR CODE HERE

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [8]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir`, defined above, as the prefix to the path.) 

**NOTES:**
- <span style='color:purple'>Leave a 2 second pause between requests</span>
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
- **Once you have written the files you can comment out this cell. This will save time and prevent you from making unnecessary requests when you restart the kernel & re-run all cells in the noteboook before submitting (as you should!)**
</div>

In [9]:
# 1.5 (5 pts) -- save each webpage to disk
for state, url in state_urls.items():
    
    # YOUR CODE HERE
    state_webpage = requests.get(url).content
    
    # writes file
    f = open(state_dir + state, 'wb')
    f.write(state_webpage)
    f.close()
    # END OF YOUR CODE HERE
    
    sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that [pandas](https://pandas.pydata.org/) is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without pandas. Thus, here we ask you to perform a few operations without using pandas, and then in Exercise 3 we will use pandas.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`
This is an example of a Python [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [10]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of [tuples](https://docs.python.org/3.3/library/stdtypes.html?highlight=tuple#tuple): (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - 7 day average case
    - 7 day average deaths
    - \# of confirmed cases (total)
    - \# of deaths
    - Stores the above 4 pieces of data above as well as <font color='green'>population</font> in a **non-pandas** data structure named `covid_data` **for every county across every state**
- Returns `covid_data`
    <font color='blue'>


**NOTES:**
- <font color='green'> Attention: the population variable not in `state_info`. More on info on where to get this value is found in the green block below</font>
- To be clear, as of September 7, 2021, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama.
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
- For the duration of our using this data for the homework, be sure to **properly store the data with the correct data types;** that is, counts should be represented as Integers and rates should be represented as Floats. For example:
    - \# of confirmed cases (total) should be an **Integer**
    - \# of deaths should be an **Integer**
    - \# of confirmed cases (per 100k) should be a **Float** (we haven't created this feature yet!)
    - 7 day average cases should be an **Integer** (you'd think an average should be a float but the values you scrapped were rounded to the nearest int)
</div>

<div style='background-color:lightgreen;padding:15px'>
    <strong>Injecting population data</strong>
    

The table on usafacts.org you've just scrapped originally had additional columns related to county population. But these have recently been removed! We'd like you to be able to utilize the population data in the following section but also use up-to-date COVID data (so the [Internet Archive](https://archive.org/) was not an option). And, though this information is available elsewhere on usafacts.org, we've decided that you've already done enough web scraping for one HW. So below we've provided a [kludge](https://en.wikipedia.org/wiki/Kludge#Computer_science).
    
`population_dict` is a nested dictionary. The keys are states whose values are _themselves_ dictionaries. Those '_inner_' dictionaries' keys are counties and their values are populations. It looks like this:
```python
{'Alabama': {'Autauga County': 55869,
             'Baldwin County': 223234,
           ...
'Wyoming': {'Albany County': 38880,
            'Big Horn County': 11790,
            ...
```

To get at a population you could use double dictionary indexing like `population_dict['Alabama']['Autauga County']`

But not all of the counties you've scrapped have population data in this dictionary. So we've provided a helper function, `get_pop`, that will return `None` if the county data was not found. Use `get_pop` to inject popoulation data into your `covid_data` as you build it up in the `load_covid_data` function you'll implement below.
    
**Final Note: you should <font color='brown'>ignore counties with missing population data or populations of 0</font>. Simply do not add them to `covid_data` as it is constructed.**
</div>

In [11]:
# load additional county population data as a nested dictionary
# you can read about this strange .pkl 'pickle' file here
# https://docs.python.org/3/library/pickle.html
with open('population.pkl', 'rb') as f:
    population_dict = pickle.load(f)

# not sure what's happening with the data types in the function header?
# check out: https://docs.python.org/3/library/typing.html#module-typing
def get_pop(state: str, county: str) -> Optional[int]:
    '''
    returns population of country, state (int)
    If county or state not found, returns None
    Example: get_pop('Alabama', 'Autauga County')
    '''
    try:
        return population_dict.get(state).get(county)
    except AttributeError:
        print('incorrect state name!')
        return None

In [29]:
def load_covid_data(state_info):
    covid_data = {}
    # YOUR CODE HERE
    for (state, state_path) in state_info:
        covid_data[state] = []
        with open(state_path, 'r') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            counties = soup.find_all('a', href=re.compile('county/'))

            for c in counties:
                row = c.find_parent('tr')

                cols = [col.text.replace(',','') for col in row.find_all('td')]

                county_name = c.text
                pop = get_pop(state, county_name)
#                 print(type(pop := get_pop(state, county_name)))
                if ((get_pop(state, county_name)) is None) or (pop == 0):
                    continue
                covid_data[state].append({'county_name': county_name,
                                          'population': pop,
                                          '7_day_avg_cases': float(cols[0]),
                                          '7_date_ave_deaths': float(cols[1]),
                                          'cases': int(cols[2]),
                                          'deaths': int(cols[3])})
    # END OF YOUR CODE HERE
    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [31]:
covid_data = load_covid_data(state_info)
covid_data

{'Alabama': [{'county_name': 'Autauga County',
   'population': 55869,
   '7_day_avg_cases': 19.0,
   '7_date_ave_deaths': 1.0,
   'cases': 9744,
   'deaths': 140},
  {'county_name': 'Baldwin County',
   'population': 223234,
   '7_day_avg_cases': 56.0,
   '7_date_ave_deaths': 3.0,
   'cases': 36447,
   'deaths': 509},
  {'county_name': 'Barbour County',
   'population': 24686,
   '7_day_avg_cases': 9.0,
   '7_date_ave_deaths': 0.0,
   'cases': 3490,
   'deaths': 71},
  {'county_name': 'Bibb County',
   'population': 22394,
   '7_day_avg_cases': 11.0,
   '7_date_ave_deaths': 0.0,
   'cases': 4131,
   'deaths': 83},
  {'county_name': 'Blount County',
   'population': 57826,
   '7_day_avg_cases': 32.0,
   '7_date_ave_deaths': 1.0,
   'cases': 9818,
   'deaths': 160},
  {'county_name': 'Bullock County',
   'population': 10101,
   '7_day_avg_cases': 2.0,
   '7_date_ave_deaths': 0.0,
   'cases': 1503,
   'deaths': 44},
  {'county_name': 'Butler County',
   'population': 19448,
   '7_day_avg

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people
   
**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any one of the tied counties (see if you can do it in an unbiased way!)
</div>

In [15]:
def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    min_county_count = 999999
    min_county_name = ""
    max_county_count = -1
    max_county_name = ""
    
    # looks through every county in every state, while checking
    # to see if we have a new low or high
    for state in covid_data.keys():
        for county in covid_data[state]:
            if ((pop := county['population']) is None) or (pop == 0):
                continue
            covid_rate = round(county['cases'] / (pop/100000),2)
            if covid_rate < min_county_count:
                min_county_count = covid_rate
                min_county_name = county['county_name'] + " (" + state + ")"
            if covid_rate > max_county_count:
                max_county_count = covid_rate
                max_county_name = county['county_name'] + " (" + state + ")"

    print(min_county_name + " has the lowest COVID cases per 100k: " + str(float(min_county_count)))
    print(max_county_name + " has the highest COVID cases per 100k: " + str(float(max_county_count)))                
                
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [16]:
calculate_county_stats(covid_data)

Lake and Peninsula Borough (Alaska) has the lowest COVID cases per 100k: 0.0
Bristol Bay Borough (Alaska) has the highest COVID cases per 100k: 72727.27


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied states

</div>

In [17]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
    min_state_deaths = 999999
    min_state_name = ""
    max_state_deaths = -1
    max_state_name = ""
    for state in covid_data.keys():
        cur_state_count = 0
        for county in covid_data[state]:
            cur_state_count += county['deaths']
            
        if cur_state_count < min_state_deaths:
            min_state_deaths = cur_state_count
            min_state_name = state
        if cur_state_count > max_state_deaths:
            max_state_deaths = cur_state_count
            max_state_name = state

    print(min_state_name + " has the fewest COVID deaths: " + str(min_state_deaths))
    print(max_state_name + " has the most COVID deaths: " + str(max_state_deaths))            
            
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [18]:
calculate_state_deaths(covid_data)

Hawaii has the fewest COVID deaths: 144
California has the most COVID deaths: 65284


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [19]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    min_state_death_rate = -1
    min_state_name = ""
    max_state_death_rate = 9999999
    max_state_name = ""
    
    for state in covid_data.keys():
        cur_state_deaths = 0
        cur_state_population = 0
        for county in covid_data[state]:
            if (county['cases'] > 0) and ((pop := county['population']) is not None):
                cur_state_population += pop
                cur_state_deaths += county['deaths']
                
        cur_state_deathrate = float(cur_state_population) / cur_state_deaths
        
        if cur_state_deathrate > min_state_death_rate:
            min_state_death_rate = cur_state_deathrate
            min_state_name = state
        if cur_state_deathrate < max_state_death_rate:
            max_state_death_rate = cur_state_deathrate
            max_state_name = state
            
    print(min_state_name + " has the lowest COVID death rate; 1 out of every " + str(round(min_state_death_rate)) + " people has died")
    print(max_state_name + " has the highest COVID death rate; 1 out of every " + str(round(max_state_death_rate)) + " people has died")
            
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [20]:
calculate_state_deathrate(covid_data)

Hawaii has the lowest COVID death rate; 1 out of every 3064 people has died
New Jersey has the highest COVID death rate; 1 out of every 334 people has died


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 5 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# covid case per 100k (Float)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 2 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [32]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    covid_data_flipped = []
    for state, counties in covid_data.items():
        for county in counties: 
            if ((pop:= county['population']) is None) or (pop == 0):
                continue
            cases = county['cases']
            cur_dict = {"county":county['county_name'], "state":state,
                            "# total covid cases": cases,
                            "# covid cases per 100k": cases/(pop/100000),
                           "# covid deaths": county['deaths']}
            covid_data_flipped.append(cur_dict)
    print(covid_data_flipped)
    covid_df = pd.json_normalize(covid_data_flipped)
    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above and inspect the results.

In [33]:
covid_df = convert_to_pandas(covid_data)

[{'county': 'Autauga County', 'state': 'Alabama', '# total covid cases': 9744, '# covid cases per 100k': 17440.79901197444, '# covid deaths': 140}, {'county': 'Baldwin County', 'state': 'Alabama', '# total covid cases': 36447, '# covid cases per 100k': 16326.814015786123, '# covid deaths': 509}, {'county': 'Barbour County', 'state': 'Alabama', '# total covid cases': 3490, '# covid cases per 100k': 14137.567852223932, '# covid deaths': 71}, {'county': 'Bibb County', 'state': 'Alabama', '# total covid cases': 4131, '# covid cases per 100k': 18446.905421094936, '# covid deaths': 83}, {'county': 'Blount County', 'state': 'Alabama', '# total covid cases': 9818, '# covid cases per 100k': 16978.521772213193, '# covid deaths': 160}, {'county': 'Bullock County', 'state': 'Alabama', '# total covid cases': 1503, '# covid cases per 100k': 14879.71487971488, '# covid deaths': 44}, {'county': 'Butler County', 'state': 'Alabama', '# total covid cases': 3203, '# covid cases per 100k': 16469.5598519127

In [23]:
covid_df.head()

,county,state,# total covid cases,# covid cases per 100k,# covid deaths
0,Autauga County,Alabama,9108,16302.421737,119
1,Baldwin County,Alabama,34393,15406.703280,400
2,Barbour County,Alabama,3225,13064.084906,65
3,Bibb County,Alabama,3694,16495.489863,74
4,Blount County,Alabama,8998,15560.474527,147


In [24]:
covid_df.shape

(3081, 5)

<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information (other than ties) as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [25]:
def calculate_county_stats2(covid_df):

    # YOUR CODE HERE
    sorted_df = covid_df.sort_values(by=['# covid cases per 100k'])
    lowest = sorted_df.iloc[0]
    highest = sorted_df.iloc[-1]

    print(f"{lowest['county']} ({lowest['state']}) has the lowest rate of confirmed COVID cases per 100k: {lowest['# covid cases per 100k']:,.2f}")
    print(f"{highest['county']} ({highest['state']}) has the highest rate of confirmed COVID cases per 100k: {highest['# covid cases per 100k']:,.2f}")
    
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [26]:
calculate_county_stats2(covid_df)

Lake and Peninsula Borough (Alaska) has the lowest rate of confirmed COVID cases per 100k: 0.00
Bristol Bay Borough (Alaska) has the highest rate of confirmed COVID cases per 100k: 72,727.27


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3 (other than ties), but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied states
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [27]:
def calculate_state_deaths2(covid_df):
    
    # YOUR CODE HERE
    state_deaths = covid_df.groupby('state').sum().sort_values(by=['# covid deaths'])
    lowest = state_deaths.iloc[0]
    highest = state_deaths.iloc[-1]

    print(lowest.name + " has the fewest COVID deaths: " + str(lowest['# covid deaths']))
    print(highest.name + " has the most COVID deaths: " + str(highest['# covid deaths']))
    
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [28]:
calculate_state_deaths2(covid_df)

Hawaii has the fewest COVID deaths: 144.0
California has the most COVID deaths: 65284.0


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.** That is, return:

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [29]:
def calculate_state_deathrate2(covid_df):
    
    # YOUR CODE HERE
    covid_df2 = covid_df
    covid_df2['population'] = 100000*covid_df2['# total covid cases'] / covid_df2['# covid cases per 100k']
    covid_df2 = covid_df2.groupby('state').sum()
    covid_df2['death_rate'] = covid_df2['population'] / covid_df2['# covid deaths']
    covid_df2 = covid_df2.sort_values(by=['death_rate'])

    print(covid_df2.iloc[-1].name + " has the lowest COVID death rate; 1 out of every " + str(int(covid_df2.iloc[-1]['death_rate'])) + " people has died")
    print(covid_df2.iloc[0].name + " has the highest COVID death rate; 1 out of every " + str(int(covid_df2.iloc[0]['death_rate'])) + " people has died")
    
    # END OF YOUR CODE HERE
    #print(____ + " has the lowest COVID death rate; 1 out of every " + ____ + " people has died")
    #print(____ + " has the highest COVID death rate; 1 out of every " + ____ + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [30]:
calculate_state_deathrate2(covid_df)

Hawaii has the lowest COVID death rate; 1 out of every 3064 people has died
New Jersey has the highest COVID death rate; 1 out of every 334 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 3 new columns:
- `case_fatality_rate`
- `# covid deaths per 100k` and
- `population`

And return the updated DataFrame **sorted by `case_fatality_rate` in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
- Make sure you inspect your results thoroughly. You may have to address the results of divisions by zero (or prevent these divisions in the first place). 
</div>

In [31]:
def add_death_stats(covid_df):
    
    # can add an infintesimal or fillna after the fact to handle nans from divide by 0.
    
    # YOUR CODE HERE
    covid_df['population'] = 100000*covid_df['# total covid cases'] / (covid_df['# covid cases per 100k']+0.0001)
#     covid_df.fillna(0, inplace=True)
    covid_df["population"] = covid_df["population"].astype('int32')
    
    covid_df['# covid deaths per 100k'] = 100000*covid_df['# covid deaths'] / (covid_df['population']+0.0001)
#     covid_df.fillna(0, inplace=True)
    covid_df["# covid deaths per 100k"] = covid_df["# covid deaths per 100k"].astype('int32')
    
    covid_df['case_fatality_rate'] =  covid_df['# covid deaths'] / (covid_df['# total covid cases']+0.0001)
#     covid_df.fillna(0, inplace=True)
    covid_df = covid_df.sort_values(by=['case_fatality_rate'])
    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [32]:
covid_updated = add_death_stats(covid_df)
covid_updated

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate
1203,Dukes County,Massachusetts,1733,9998.846065,0,17331,0,0.000000
1689,Logan County,Nebraska,82,10962.566845,0,747,0,0.000000
2651,Loving County,Texas,7,4142.011834,0,168,0,0.000000
2782,Wayne County,Utah,177,6528.956105,0,2710,0,0.000000
1690,Loup County,Nebraska,42,6325.301205,0,663,0,0.000000
...,...,...,...,...,...,...,...,...
417,Dodge County,Georgia,1396,6775.054598,113,20604,548,0.080946
528,Wilcox County,Georgia,597,6913.723219,49,8634,567,0.082077
1670,Grant County,Nebraska,41,6581.059390,4,622,643,0.097561
434,Glascock County,Georgia,171,5755.637832,19,2970,639,0.111111


<div class='exercise'><b>Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6.1 [2 pts] Trends</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**

States vary wildly in their death rate (e.g., The number of deaths in New Jersey or California is orders of magnitude higher than those in Hawaii or Alaska) and COVID testing -- evident by the large variance in the `case_fatality_rate`. This is exacerbated by many counties having small populations and/or few reported deaths. States also fluctuate a lot amongst their counties, as some counties with extremely bad statistics are within states that at large at fairing better.
</div>

<div class='exercise'><b>Exercise 3.6.2 [2 pts]: Data Reliability</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**

It is possible that some states and counties are more proactive when it comes to testing. This could result in higher cases counts. Other counties might have a similar number of cases or higher, but they are just not being represented in the data due to lower testing. Deaths on the other hand are harder to overlook in this way, so states with lax testing policies may have inflated death per case metrics. Perhaps we could suppliment the data with some measure of testing rates in the county or state. 
    
</div>

<div class='exercise'><b>Exercise 3.6.3 [1 pt]: Relationships Between Variables</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**
    
For our limited look at the data above we could use a rule of thumb regarding the apparent relation between deaths, cases, and population. Eyeballing the table above we see that in many counties the number of cases is roughly an order of magnitute higher than the namber of death and that the population is itself roughly 200x the number of deaths (or atleast in that order of magnitude). We could be more principled and instead use the mean death rate per 100k and mean fatality rate per 100k to calculate the expected cases and population from deahts, but even this is a crude estimate. To continue we'd like to see how these relationships vary depending on the number of deaths as the multiples are likely not constant across all counties and the relationships may be very different for counties with very few deaths compared to those with many.

</div>

<div class='exercise'><b>Exercise 3.6.4 [1 pt]: Further Questions</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**

Many possible questions for this one but here are some:

- What is the covid testing rate in these counties?
- Is there a correlation between income level and death rate?
- What are the counties access to health care? (people with insurance, hospital capacity, ect.)
    
</div>

## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2020_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2020_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 8 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- \# covid deaths per 100k
- case_fatality_rate

along with these 15 columns from `election2020_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- voter_turnout
- voter_gap
- trump
- biden

**NOTES:**
- We are dropping two columns from `election2020_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful

**EXTRA INFORMATION:** In case you're wondering what the different features/columns are in `election2020_by_county.csv`:

- state: the state in which the county lies
- fipscode: an ID to identify each county
- county: the name of each county
- population: total population
- hispanic: percent of adults that are hispanic
- minority: percent of adults that are nonwhite
- female: percent of adults that are female
- unemployed: unemployment rate, as a percent
- income: median income
- nodegree: percent of adults who have not completed high school
- bachelor: percent of adults with a bachelor’s degree
- inactive: percent of adults who do not exercise in their leisure time
- obesity: percent of adults with BMI > 30
- density: population density, persons per square mile of land
- cancer: prevalence of cancer per 100,000 individuals
- voter_turnout: percentage of voting age population that voted
- voter_gap: percentage point gap in 2020 presidential voting: trump-briden
</div>

In [33]:
def merge_data(covid_updated, filepath):
    
    # YOUR CODE HERE
    data2020 = pd.read_csv(filepath).drop(columns=['fipscode', 'population'])
    return pd.merge(covid_updated, data2020, on=['state', 'county'])
    # END OF YOUR CODE HERE
    #return ____

Run the cell below (no changes necessary) to execute your code above

In [34]:
merged = merge_data(covid_updated, 'election2020_by_county.csv')

In [35]:
merged.head()

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
0,Dukes County,Massachusetts,1733,9998.846065,0,17331,0,0.0,1.6,13.0,...,6.6,41.2,17.7,22.0,991.3,209.3,NaN,NaN,NaN,NaN
1,Logan County,Nebraska,82,10962.566845,0,747,0,0.0,2.9,6.1,...,8.1,20.4,28.4,31.1,14.2,216.6,30.124224,82.0,90.4,8.4
2,Loving County,Texas,7,4142.011834,0,168,0,0.0,16.2,25.7,...,11.8,2.6,21.9,29.0,20.7,357.8,-11.864407,84.8,90.9,6.1
3,Wayne County,Utah,177,6528.956105,0,2710,0,0.0,5.4,8.2,...,5.7,26.5,21.0,22.8,56.9,153.8,15.911486,52.9,75.6,22.7
4,Loup County,Nebraska,42,6325.301205,0,663,0,0.0,0.0,0.4,...,6.2,14.4,33.0,30.7,1.3,NaN,-4.849885,65.0,81.5,16.5


In [36]:
merged.shape

(3075, 23)

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2.1 [1 pt]: Lost Rows</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [37]:
# YOUR CODE HERE
print(len(covid_updated) - len(merged))
# END OF YOUR CODE HERE

6


<div class='exercise'><b>Exercise 4.2.2 [2 pts]: Lost Counties</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [38]:
# YOUR CODE HERE
missing_counties = set()
merged_counties = set()
for index, row in merged.iterrows():
    merged_counties.add(row['county'].lower() + "_" + row['state'].lower())

missing_idxs = []
for index, row in covid_updated.iterrows():
    cur_county = row['county'].lower() + "_" + row['state'].lower()
    if cur_county not in merged_counties:
        print("missing",cur_county)
        missing_idxs.append(index)
        missing_counties.add(cur_county)
# END OF YOUR CODE HERE

missing kalawao county_hawaii
missing kusilvak census area_alaska
missing aleutians west census area_alaska
missing southeast fairbanks census area_alaska
missing doña ana county_new mexico
missing oglala lakota county_south dakota


In [39]:
# Missing Counties
covid_updated.loc[missing_idxs]

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate
533,Kalawao County,Hawaii,0,0.000000,0,0,0,0.000000
78,Kusilvak Census Area,Alaska,1508,18138.080346,4,8313,48,0.002653
68,Aleutians West Census Area,Alaska,265,4703.585375,1,5633,17,0.003774
84,Southeast Fairbanks Census Area,Alaska,799,11591.469607,7,6892,101,0.008761
1780,Doña Ana County,New Mexico,27452,12581.406540,509,218194,233,0.018541
2390,Oglala Lakota County,South Dakota,2192,15461.663257,51,14176,359,0.023266


<div class='exercise'><b>Exercise 4.2.3 [2 pts]: Suggested Fixes</b>
   
If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**
    
In the original version of the population pickle file, matches fail for Louisiana. This is due to these counties have "parish" appended to their names on the website, while "parish" is dropped in the naming convention used in the population data (even though both came fron USAFacts.org!).
    Most of the other mismatches are due to the 2020 dataset referring to some counties as "county, city of", whereas the USAFacts.org site simply uses "county". We could use a Regular Expression to replace these; yet, the other mistakes are due to Unicode (e.g., $&#x27;$) and referring to counties as "census area".
This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

</div>

This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [40]:
# YOUR CODE HERE
merged = merged.loc[merged['# covid deaths'] != 0]
# END OF YOUR CODE HERE

Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [41]:
merged.describe()

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,3.041000e+03,3041.000000,3041.000000,3.041000e+03,3041.000000,3041.000000,3040.000000,3040.000000,3040.000000,3041.000000,...,3041.000000,3041.000000,3041.000000,3041.000000,3041.000000,3001.000000,3009.000000,3008.000000,3008.000000,3008.000000
mean,1.275353e+04,12459.248573,205.926998,1.056227e+05,226.143374,0.018477,9.149013,22.857928,49.919308,5.525551,...,15.099474,19.905130,26.080697,31.105886,229.098619,228.372009,35.578751,32.994149,65.638630,32.644481
std,4.332138e+04,3630.207220,768.170288,3.374344e+05,118.254022,0.009800,13.820543,19.824572,2.380035,1.967128,...,6.763892,8.707507,5.177858,4.484579,1695.636176,55.510932,13.694091,30.816176,15.470732,15.354665
min,1.400000e+01,2220.131963,1.000000,4.030000e+02,5.000000,0.000978,0.000000,0.200000,19.166215,1.800000,...,1.900000,4.400000,8.100000,11.800000,0.100000,46.200000,-168.323353,-90.000000,4.000000,3.100000
25%,1.379000e+03,10205.971249,23.000000,1.139100e+04,142.000000,0.012371,2.000000,7.000000,49.466600,4.200000,...,10.000000,13.900000,22.800000,28.400000,17.700000,193.500000,27.810525,15.550000,56.875000,20.900000
50%,3.273000e+03,12411.755164,57.000000,2.642600e+04,211.000000,0.016729,3.900000,15.500000,50.398389,5.300000,...,13.600000,17.800000,25.900000,31.300000,45.100000,230.200000,35.162141,39.100000,68.700000,29.600000
75%,8.467000e+03,14640.671536,137.000000,6.943100e+04,290.000000,0.022732,9.300000,34.500000,51.088533,6.500000,...,19.400000,23.500000,29.600000,33.900000,110.700000,264.600000,42.478361,56.725000,77.500000,41.500000
max,1.360180e+06,72727.272727,25483.000000,1.003911e+07,866.000000,0.142856,99.200000,99.400000,56.633907,24.000000,...,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,100.000000,93.100000,96.200000,94.000000


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [42]:
def partition_df(df, column_name, minv, maxv):
    # YOUR CODE HERE
    return df.loc[(merged[column_name] >= minv) & (merged[column_name] <= maxv)]
    # END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [43]:
# YOUR CODE HERE
partition_df(merged, 'income', 21000, 31000).describe()
# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,...,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000
mean,2495.933884,14093.049503,54.528926,16677.603306,319.975207,0.023939,8.215702,44.409091,49.551705,9.118182,...,26.886777,11.815702,31.850413,36.371901,83.109917,241.257025,44.908995,13.433058,56.076860,42.643802
std,3023.099014,3809.136598,65.744718,17881.233436,140.086965,0.012134,20.992959,32.024454,3.863055,2.245291,...,5.672785,3.637857,4.331688,5.373813,158.592921,48.314351,10.795226,45.084074,22.579658,22.512089
min,112.000000,4871.683341,2.000000,1536.000000,26.000000,0.001691,0.000000,0.400000,35.462777,4.400000,...,8.000000,5.800000,19.700000,21.000000,1.400000,99.200000,-10.411765,-71.600000,13.500000,9.300000
25%,968.000000,11502.408541,19.000000,8063.000000,216.000000,0.016431,0.700000,6.400000,48.747893,7.700000,...,23.700000,9.300000,29.100000,33.300000,18.000000,212.200000,39.269619,-27.000000,35.600000,20.400000
50%,1618.000000,14364.214788,36.000000,11194.000000,318.000000,0.022340,1.600000,50.500000,50.654499,8.700000,...,26.400000,11.200000,32.200000,36.400000,37.600000,239.900000,44.536665,13.500000,54.900000,42.700000
75%,2574.000000,16428.224370,55.000000,18522.000000,406.000000,0.026954,3.100000,72.800000,51.918536,10.300000,...,29.800000,13.600000,35.200000,40.300000,75.500000,272.300000,51.030085,57.800000,78.000000,62.600000
max,22506.000000,26696.123147,489.000000,130624.000000,839.000000,0.073958,99.200000,99.400000,56.526573,17.600000,...,53.300000,27.900000,41.300000,47.600000,996.700000,380.000000,70.003709,80.400000,89.700000,85.100000


In [44]:
z = merged.cancer.quantile(0.5)

In [45]:
# YOUR CODE HERE
def view_partitions(df, feature, n_partitions=3, cols=None):
    if cols is None:
        cols = df.columns
    start = 0
    for  i in range(n_partitions):
        stop = start + (1/n_partitions)
        display(partition_df(merged, feature,
                             merged[feature].quantile(start),
                             merged[feature].quantile(stop))[cols].describe())
        start = stop
view_partitions(merged, 'obesity')
# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1.024000e+03,1024.000000,1024.000000,1.024000e+03,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,...,1024.000000,1024.000000,1024.000000,1024.000000,1024.000000,1005.000000,1008.000000,1008.000000,1008.000000,1008.000000
mean,2.301558e+04,11245.026814,359.409180,1.977969e+05,193.865234,0.017567,14.867969,24.807617,49.772990,5.053027,...,13.071387,25.463184,22.147559,26.325781,318.360840,213.755323,32.575558,24.924702,61.539683,36.614980
std,7.038826e+04,3679.174439,1242.825454,5.435931e+05,122.284260,0.011406,18.063739,19.903407,2.489231,1.836033,...,6.759016,10.718269,4.219824,2.963034,1528.473747,60.398586,17.587883,35.855241,18.042064,17.822056
min,1.400000e+01,2220.131963,1.000000,4.030000e+02,5.000000,0.001076,0.000000,0.200000,31.955024,1.800000,...,1.900000,6.900000,8.100000,11.800000,0.100000,46.200000,-13.483146,-90.000000,4.000000,3.100000
25%,1.137500e+03,8827.975888,19.000000,1.142550e+04,105.750000,0.010494,3.100000,9.500000,49.349121,3.800000,...,8.400000,17.275000,19.300000,25.100000,11.300000,170.500000,21.639556,-1.500000,48.400000,21.900000
50%,3.668500e+03,11140.682215,55.000000,3.488400e+04,166.000000,0.015124,7.350000,18.400000,50.329872,4.800000,...,11.300000,23.300000,22.500000,27.250000,45.650000,212.600000,29.940663,27.950000,63.000000,34.950000
75%,1.623650e+04,13547.379257,223.250000,1.575675e+05,257.000000,0.021820,19.225000,34.775000,51.005636,5.900000,...,16.225000,32.100000,25.000000,28.500000,154.850000,251.800000,40.189642,54.350000,76.300000,49.525000
max,1.360180e+06,40267.717979,25483.000000,1.003911e+07,866.000000,0.142856,95.300000,97.400000,55.707947,24.000000,...,46.800000,72.000000,38.800000,29.500000,32903.300000,433.900000,99.552895,93.100000,96.200000,94.000000


,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1043.000000,1043.000000,1043.000000,1.043000e+03,1043.000000,1043.000000,1043.000000,1043.000000,1043.000000,1043.000000,...,1043.000000,1043.000000,1043.000000,1043.000000,1043.000000,1032.000000,1035.000000,1034.000000,1034.000000,1034.000000
mean,9040.499521,12536.253580,144.679770,7.197545e+04,228.105465,0.018578,7.265292,18.539789,49.921429,5.333365,...,14.535187,18.574401,26.413615,31.290125,198.329434,234.883527,34.985290,38.686170,68.489168,29.802998
std,18409.286865,3125.094452,330.510553,1.467280e+05,114.639627,0.009542,11.172982,16.855867,2.199235,1.838091,...,6.145999,6.142060,3.971328,1.042130,2183.305210,54.034435,12.455518,24.682625,12.384693,12.307151
min,22.000000,4638.174288,1.000000,4.620000e+02,14.000000,0.000978,0.000000,1.500000,19.166215,1.800000,...,3.800000,4.400000,16.400000,29.500000,0.200000,82.500000,-168.323353,-68.100000,15.000000,5.600000
25%,1268.500000,10607.128766,22.500000,1.055200e+04,149.000000,0.012535,1.900000,6.000000,49.415479,4.100000,...,10.000000,14.200000,23.500000,30.400000,17.800000,200.050000,28.427332,23.900000,60.900000,20.400000
50%,3218.000000,12489.642185,57.000000,2.640300e+04,208.000000,0.016589,3.400000,12.000000,50.346270,5.200000,...,13.100000,17.500000,26.300000,31.300000,43.300000,236.350000,34.629388,42.150000,70.250000,28.000000
75%,7957.000000,14424.082575,129.000000,6.446350e+04,284.000000,0.022275,7.250000,26.800000,50.990937,6.400000,...,18.150000,21.600000,28.900000,32.200000,98.600000,268.925000,40.833752,57.475000,77.900000,37.100000
max,190914.000000,35687.475306,6647.000000,1.584063e+06,839.000000,0.082077,99.200000,99.400000,56.633907,21.800000,...,53.300000,43.700000,41.400000,33.000000,69468.400000,425.400000,100.000000,86.000000,92.000000,83.100000


,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1027.000000,1027.000000,1027.000000,1.027000e+03,1027.000000,1027.000000,1026.000000,1026.000000,1026.000000,1027.000000,...,1027.000000,1027.000000,1027.000000,1027.000000,1027.000000,1017.000000,1019.000000,1019.000000,1019.000000,1019.000000
mean,6134.600779,13625.198267,112.320351,4.647348e+04,256.364167,0.019248,5.309942,25.028558,50.077660,6.183155,...,17.698734,15.628140,29.711977,35.711100,166.793768,236.452901,39.239150,35.387537,66.894504,31.506968
std,11315.165400,3665.205147,253.309613,9.404056e+04,107.590914,0.007975,8.432645,21.666278,2.393791,2.034852,...,6.487851,4.782821,4.205198,2.414385,1157.566205,48.284292,8.658075,28.942876,14.471159,14.481409
min,112.000000,4011.632242,1.000000,8.350000e+02,14.000000,0.000978,0.000000,0.400000,33.597815,1.800000,...,3.100000,5.800000,15.600000,33.000000,0.100000,61.800000,12.181303,-80.800000,8.800000,5.400000
25%,1678.500000,11342.701941,31.000000,1.279050e+04,184.000000,0.014074,1.600000,6.400000,49.634400,4.900000,...,12.700000,12.200000,26.750000,33.900000,23.600000,205.700000,33.382109,21.000000,59.900000,21.000000
50%,3202.000000,13563.402889,58.000000,2.343600e+04,245.000000,0.017931,2.800000,17.200000,50.532720,6.000000,...,17.100000,14.800000,29.800000,35.100000,46.700000,236.600000,39.391691,42.000000,70.100000,28.200000
75%,6172.500000,15707.152959,112.000000,4.593200e+04,315.000000,0.023220,5.200000,39.800000,51.282194,7.300000,...,22.400000,18.400000,32.800000,36.900000,101.800000,265.900000,44.972609,56.550000,77.400000,38.450000
max,178417.000000,72727.272727,5331.000000,1.749342e+06,713.000000,0.065217,91.800000,93.400000,56.526573,16.900000,...,40.500000,42.600000,41.300000,47.600000,35369.200000,458.300000,70.003709,87.900000,93.300000,89.600000


In [46]:
view_partitions(merged, 'inactivity')

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1.032000e+03,1032.000000,1032.000000,1.032000e+03,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,...,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1018.000000,1011.000000,1011.000000,1011.000000,1011.000000
mean,2.314645e+04,11150.432527,340.902132,2.025585e+05,169.793605,0.015325,12.649031,22.909012,49.690513,4.983818,...,11.510853,26.247674,20.551744,27.851066,403.765407,214.263065,31.538778,18.227201,58.094461,39.867260
std,6.826995e+04,3931.830558,1145.262913,5.294841e+05,105.646112,0.008038,16.398927,19.055224,2.457474,1.796694,...,5.894800,10.231851,2.863869,4.197408,2668.750016,60.503159,17.027280,32.219850,16.146636,16.081957
min,5.600000e+01,2220.131963,1.000000,4.030000e+02,5.000000,0.001076,0.000000,0.600000,19.166215,1.900000,...,1.900000,7.500000,8.100000,11.800000,0.300000,46.200000,-5.066079,-90.000000,4.000000,3.100000
25%,1.671250e+03,8951.968278,22.000000,1.607500e+04,100.000000,0.010100,3.000000,8.600000,49.321641,3.800000,...,7.800000,18.500000,18.800000,25.400000,12.500000,168.450000,21.468618,-4.550000,46.500000,27.900000
50%,5.049000e+03,11095.477071,71.000000,4.834000e+04,149.000000,0.013807,5.800000,15.850000,50.206483,4.700000,...,10.100000,24.400000,21.400000,28.300000,46.000000,212.350000,28.493636,21.100000,59.300000,38.400000
75%,1.874150e+04,13125.103461,246.000000,1.685010e+05,213.250000,0.018780,14.425000,31.750000,50.831317,5.800000,...,13.200000,32.225000,22.800000,30.800000,159.975000,254.675000,37.809084,42.200000,70.000000,51.050000
max,1.360180e+06,72727.272727,25483.000000,1.003911e+07,796.000000,0.058166,95.300000,97.400000,54.555681,24.000000,...,43.900000,72.000000,23.900000,39.700000,69468.400000,397.400000,99.552895,93.100000,96.200000,94.000000


,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1038.000000,1038.000000,1038.000000,1.038000e+03,1038.000000,1038.000000,1037.000000,1037.000000,1037.000000,1038.000000,...,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1020.00000,1027.000000,1026.000000,1026.000000,1026.000000
mean,9814.897881,12397.532831,179.136802,7.644492e+04,244.025048,0.020296,9.974542,22.274060,49.927892,5.249037,...,14.871965,18.671676,26.005780,31.134875,171.071195,229.32549,34.874362,38.247953,68.315010,30.067057
std,24924.775275,3282.397185,585.472339,1.914381e+05,122.417561,0.011143,15.056662,20.258563,2.239983,1.851616,...,6.292695,5.845579,1.244686,3.048128,1120.419615,53.42843,12.657893,26.835066,13.461418,13.383122
min,14.000000,2240.000000,1.000000,4.860000e+02,14.000000,0.001495,0.000000,0.200000,32.813627,1.800000,...,3.100000,4.400000,23.900000,22.400000,0.100000,70.00000,-168.323353,-70.900000,14.100000,3.900000
25%,1059.500000,10274.425422,20.000000,9.125500e+03,158.250000,0.013357,2.100000,6.200000,49.376147,4.100000,...,10.600000,14.500000,25.000000,28.825000,14.200000,195.07500,28.556416,22.800000,60.600000,20.300000
50%,2712.500000,12359.778588,51.500000,2.223200e+04,225.000000,0.018193,4.100000,14.200000,50.333745,5.050000,...,13.400000,17.900000,25.900000,31.100000,41.100000,230.25000,34.565739,41.450000,69.900000,28.300000
75%,6977.500000,14401.442911,120.750000,5.816675e+04,302.750000,0.024837,10.100000,34.500000,51.059021,6.200000,...,17.875000,21.600000,27.000000,33.200000,106.425000,263.00000,40.957131,58.100000,78.275000,37.700000
max,310776.000000,40267.717979,10644.000000,2.559902e+06,866.000000,0.142856,99.200000,99.400000,56.633907,21.800000,...,53.300000,46.300000,28.200000,42.000000,32903.300000,433.90000,100.000000,92.000000,95.900000,85.000000


,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1017.000000,1017.000000,1017.000000,1.017000e+03,1017.000000,1017.000000,1017.00000,1017.000000,1017.000000,1017.000000,...,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,1009.000000,1016.000000,1016.000000,1016.000000,1016.000000
mean,5089.915438,13821.796675,95.706981,3.614396e+04,265.166175,0.019868,4.84297,23.325860,50.135491,6.345821,...,18.917109,14.695084,31.762045,34.400983,108.645821,241.338057,40.205570,42.465059,70.485925,28.020866
std,9346.468949,3113.089372,243.825529,6.689032e+04,103.371915,0.009118,7.16761,20.211823,2.396202,1.977488,...,5.945731,4.278158,2.675647,3.425351,223.453651,48.184123,8.440751,27.342446,13.656017,13.695339
min,22.000000,4751.619870,1.000000,4.620000e+02,14.000000,0.000978,0.00000,0.400000,33.597815,1.800000,...,3.900000,5.800000,28.200000,23.800000,0.300000,94.400000,1.730104,-71.600000,13.500000,5.000000
25%,1473.000000,11717.253202,28.000000,1.113600e+04,195.000000,0.014178,1.50000,6.000000,49.773707,5.100000,...,14.600000,11.700000,29.600000,32.100000,25.100000,211.200000,34.533637,32.075000,65.375000,18.800000
50%,2886.000000,13842.096308,55.000000,2.083600e+04,257.000000,0.018113,2.60000,16.100000,50.631215,6.200000,...,18.700000,14.100000,31.300000,34.100000,46.700000,240.500000,40.027305,50.000000,74.100000,24.300000
75%,5756.000000,15847.694012,104.000000,4.036600e+04,328.000000,0.023894,4.90000,36.500000,51.316339,7.400000,...,22.900000,17.000000,33.500000,36.500000,97.000000,270.500000,45.843751,60.800000,79.700000,33.025000
max,190914.000000,27480.036855,6647.000000,1.418206e+06,711.000000,0.097561,63.50000,91.200000,56.526573,16.900000,...,40.500000,36.900000,41.400000,47.600000,2800.000000,458.300000,70.003709,88.300000,93.300000,85.100000


In [47]:
view_partitions(merged, 'income',
                cols=['income','# total covid cases', '# covid cases per 100k', '# covid deaths',
                      'population', '# covid deaths per 100k', 'case_fatality_rate',
                      'obesity', 'inactivity', 'trump', 'biden'])

,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity,trump,biden
count,1014.000000,1014.000000,1014.000000,1014.000000,1.014000e+03,1014.000000,1014.000000,1014.000000,1014.000000,1010.000000,1010.000000
mean,35791.410256,4564.531558,13350.222242,94.304734,3.348277e+04,280.529586,0.022066,33.135503,29.494379,67.547624,31.031683
std,3791.473990,11139.018138,3816.829098,291.116740,8.538418e+04,123.871501,0.011772,4.464598,4.660334,15.820049,15.763743
min,21658.000000,14.000000,2240.000000,1.000000,4.030000e+02,14.000000,0.001562,17.400000,13.600000,13.500000,7.100000
25%,33431.500000,1201.250000,10806.661455,25.000000,9.886250e+03,193.000000,0.014871,30.300000,26.000000,58.625000,18.900000
50%,36443.000000,2387.500000,13463.912711,49.000000,1.790200e+04,272.000000,0.019555,33.150000,29.800000,72.200000,26.300000
75%,38775.250000,4560.000000,15759.901260,89.000000,3.222550e+04,351.000000,0.026502,36.200000,33.000000,79.600000,39.900000
max,41079.000000,190914.000000,36941.098829,6647.000000,1.584063e+06,866.000000,0.142856,47.600000,41.400000,92.600000,85.100000


,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity,trump,biden
count,1013.000000,1013.000000,1013.000000,1013.000000,1.013000e+03,1013.000000,1013.000000,1013.000000,1013.000000,1004.000000,1004.000000
mean,45253.674235,10388.895360,12226.777226,165.553801,8.051626e+04,215.939783,0.017879,31.074729,25.879566,67.863247,30.370319
std,2600.690989,29461.161835,3248.983662,498.445314,1.909502e+05,104.548972,0.008390,3.733559,4.210464,12.911955,12.775929
min,41085.000000,22.000000,3164.384309,1.000000,4.620000e+02,11.000000,0.001076,14.800000,11.200000,14.100000,5.000000
25%,43040.000000,1298.000000,10200.220173,22.000000,1.113000e+04,140.000000,0.012405,28.800000,23.200000,60.675000,21.000000
50%,45151.000000,3444.000000,12092.698708,58.000000,2.920900e+04,210.000000,0.016533,31.300000,25.900000,70.300000,27.900000
75%,47381.000000,7765.000000,14285.714286,134.000000,6.469600e+04,273.000000,0.022242,33.500000,28.600000,77.300000,37.600000
max,50014.000000,632579.000000,40267.717979,10644.000000,2.716939e+06,795.000000,0.097561,42.100000,38.600000,94.000000,85.000000


,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity,trump,biden
count,1014.000000,1.014000e+03,1014.000000,1014.000000,1.014000e+03,1014.000000,1014.000000,1014.000000,1014.000000,994.000000,994.000000
mean,59768.488166,2.330483e+04,11800.516990,357.882643,2.028443e+05,181.950690,0.015484,29.107396,22.867949,61.451911,36.580282
std,10586.899122,6.675883e+04,3626.506252,1183.500199,5.316773e+05,103.354657,0.007557,4.286441,4.357840,16.607793,16.528661
min,50020.000000,2.900000e+01,2220.131963,1.000000,4.930000e+02,5.000000,0.000978,11.800000,8.100000,4.000000,3.100000
25%,52415.250000,1.758000e+03,9734.487107,24.000000,1.511775e+04,114.250000,0.010482,27.000000,20.300000,51.100000,24.725000
50%,56020.000000,5.624500e+03,11874.547426,79.500000,5.023400e+04,165.000000,0.014163,29.500000,23.050000,63.250000,34.700000
75%,62435.750000,1.943025e+04,13802.075414,261.000000,1.703000e+05,230.000000,0.018935,32.200000,25.900000,73.350000,46.700000
max,122641.000000,1.360180e+06,72727.272727,25483.000000,1.003911e+07,796.000000,0.058823,40.200000,37.800000,96.200000,94.000000


In [48]:
view_partitions(merged, 'trump',
                cols=['trump','income','# total covid cases', '# covid cases per 100k', '# covid deaths',
                      'population', '# covid deaths per 100k', 'case_fatality_rate',
                      'obesity', 'inactivity',])

,trump,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity
count,1004.000000,1004.000000,1.004000e+03,1004.000000,1004.000000,1.004000e+03,1004.000000,1004.000000,1004.000000,1004.000000
mean,47.700000,49957.554781,2.867887e+04,11631.053349,455.145418,2.433075e+05,200.549801,0.017098,29.970817,23.538347
std,11.162241,15303.487288,7.186218e+04,3648.336352,1282.862007,5.546238e+05,120.413928,0.008903,5.696659,5.515891
min,4.000000,21658.000000,1.020000e+02,2220.131963,1.000000,7.680000e+02,5.000000,0.001076,11.800000,8.100000
25%,40.900000,38876.250000,2.427000e+03,9320.124427,43.750000,2.241350e+04,114.750000,0.011124,26.300000,19.800000
50%,50.400000,48229.000000,7.546500e+03,11665.015340,107.500000,6.977750e+04,174.000000,0.015449,30.000000,23.300000
75%,56.900000,57095.500000,2.636475e+04,13835.271535,367.000000,2.344940e+05,262.000000,0.020823,33.400000,26.825000
max,61.300000,122641.000000,1.360180e+06,40267.717979,25483.000000,1.003911e+07,839.000000,0.073958,47.600000,40.800000


,trump,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity
count,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000,1008.000000
mean,68.404167,46773.963294,6680.970238,12746.934355,108.261905,51149.239087,229.904762,0.018631,31.786706,26.442460
std,3.832817,9320.739632,9480.152069,3315.844992,150.655018,66676.213800,109.632214,0.010036,3.579210,4.228102
min,61.300000,25768.000000,14.000000,2240.000000,1.000000,403.000000,11.000000,0.000978,19.500000,14.100000
25%,65.200000,40400.000000,1640.250000,10549.995638,29.000000,13686.750000,149.000000,0.012516,29.500000,23.375000
50%,68.700000,45662.500000,3713.000000,12833.331581,63.000000,28655.500000,214.000000,0.016767,31.900000,26.200000
75%,71.700000,51796.000000,7296.750000,14862.345045,127.500000,60505.500000,290.000000,0.022315,34.200000,29.300000
max,74.500000,97936.000000,111719.000000,36941.098829,2182.000000,636234.000000,795.000000,0.142856,42.300000,39.900000


,trump,income,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,obesity,inactivity
count,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000
mean,80.768775,43782.178854,2811.449605,13062.377686,50.146245,20564.411067,251.620553,0.019852,31.608202,28.392095
std,4.277882,9003.383179,3442.561533,3170.983250,61.268461,23517.040977,118.730827,0.010172,3.596872,4.505372
min,74.500000,23047.000000,14.000000,2874.743326,1.000000,462.000000,14.000000,0.001562,19.400000,13.600000
25%,77.500000,37405.500000,740.250000,10924.498794,13.000000,6040.750000,169.750000,0.013567,29.075000,25.100000
50%,80.100000,42132.500000,1757.000000,12952.910860,31.000000,13604.000000,238.000000,0.017746,31.600000,28.000000
75%,83.600000,49036.250000,3609.500000,15108.120682,66.000000,26202.250000,309.000000,0.024481,33.900000,32.000000
max,96.200000,86354.000000,34393.000000,24697.221563,862.000000,223233.000000,866.000000,0.111111,43.200000,41.400000


<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**

Above we split the data into equal lower, middle, and upper quantiles based on first `obeity` and then `inactivity`. We can see that the the average death rates of counties in these partitions is positivly correlated with both of these features. This was expected as preexisting health conditions (obescity) and heath risks (inactivity) increase all cause mortality but also have a strong effect on how serious a covid infection can be. Finally we see that income has an even stronger relationship with the death rate, though here the correlation is a negative one. Obesity and inactivity are both negatively correlated with income as well. The relationship between voting for Trump and income is not a string one strong, though there is a positive correlation between Trump voting and obesity, inactivity, and covid death rate. We are of course not making any causal claims here as these are unjustified outside of a controlled experiment.
    
</div>

`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [49]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

,county,state,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,...,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden,income group
34,Saline County,Nebraska,2044,14370.078740,2,14223,14,0.000978,24.3,29.3,...,14.3,31.0,33.0,1.5,309.1,40.293874,28.5,62.9,34.4,income-group-3
35,Lake County,Colorado,929,11431.032361,1,8126,12,0.001076,33.9,35.7,...,30.3,15.1,17.5,19.4,112.4,29.853937,-20.2,37.9,58.1,income-group-3
36,Dodge County,Minnesota,2311,11039.457342,3,20933,14,0.001298,4.9,7.5,...,24.1,18.3,24.9,709.0,140.3,17.427640,30.5,64.0,33.5,income-group-4
37,Pitkin County,Colorado,2868,16142.286261,4,17766,22,0.001395,9.8,14.3,...,56.4,8.9,14.9,17.7,70.8,12.694664,-52.1,23.2,75.3,income-group-4
38,Jefferson County,Nebraska,669,9494.748794,1,7045,14,0.001495,3.6,5.8,...,13.2,28.0,36.2,11.0,230.7,33.996448,43.1,70.4,27.3,income-group-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3070,Dodge County,Georgia,1396,6775.054598,113,20604,548,0.080946,3.4,34.7,...,14.4,28.8,28.5,44.0,212.3,51.126453,45.5,72.4,26.9,income-group-1
3071,Wilcox County,Georgia,597,6913.723219,49,8634,567,0.082077,4.3,41.5,...,8.7,27.7,31.5,24.5,248.4,53.295374,46.9,73.2,26.3,income-group-1
3072,Grant County,Nebraska,41,6581.059390,4,622,643,0.097561,1.9,3.8,...,18.6,30.6,28.3,4.5,305.1,23.574144,88.3,93.3,5.0,income-group-3
3073,Glascock County,Georgia,171,5755.637832,19,2970,639,0.111111,1.6,12.6,...,11.4,24.8,28.6,21.4,264.6,32.529082,79.7,89.6,9.9,income-group-2


<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [50]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby('income group').mean()
# END OF YOUR CODE HERE

,# total covid cases,# covid cases per 100k,# covid deaths,population,# covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
income group,,,,,,,,,,,,,,,,,,,,,
income-group-1,3914.182085,13615.466077,85.320117,28207.182085,294.240822,0.022849,8.870044,32.753891,49.980281,7.318502,...,22.047430,13.603671,30.212041,33.792952,90.745521,237.447788,41.772286,32.861062,65.754867,32.893805
income-group-2,8520.437576,12489.246427,144.338182,64748.787879,235.003636,0.019232,8.779126,20.646966,49.821097,5.738788,...,15.688485,17.509455,27.076364,31.584606,132.777818,234.975124,37.150281,40.767722,69.566991,28.799269
income-group-3,12775.409524,12310.722884,193.202721,99245.555102,208.519728,0.017124,8.771293,18.513061,49.886102,4.891429,...,12.912925,20.371020,25.205306,30.736735,336.364762,227.790884,34.352153,35.464835,66.802198,31.337363
income-group-4,24623.295000,11580.541113,383.797500,219532.907500,175.230000,0.015219,10.114500,20.703125,49.999070,4.362000,...,10.586500,27.311750,22.341375,28.664000,347.651750,214.379646,29.685445,22.634955,60.323560,37.688604


<div class='exercise'><b>Wrapping Up</b>
</div>

<div class='exercise'><b>Exercise 4.7.1 [1 pt]: Conclusions</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**
    
We notice that the covid deaths and cases per 100k are much higher in the bottom income-group-1 but don't change as significantly between the other 3 groups. The lower income group has much higher rates of obesity, inactivity, and unemployment which might also contribute covid mortality. The lower 2 income groups also has much less population density than the higher 2, so they are situated in more rural areas. There is also a big gap in minority population between income group 1 and the other groups.

</div>

<div class='exercise'><b>Exercise 4.7.2 [1 pt]: Possible Weaknesses</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**
    
We can tell from comparing the populations between the groups that this data is not treated granularly as would be ideal. Very small population counties that get weighted the same as very large population counties in regards to the mean. So rural areas get over represented in the averages nationwide. This also explains why the Trump vs Biden is so far skewed from the actual well known national average based on popular votes.

Also, the difference between income correlates more with population density than it might with an individual socio economic status. First, a higher income might not go as far towards standard of living in the city as it does in rural areas. Second, by using the average income over the whole county, income inequality in that county is not factored in. There could be many low income individuals living with many high income individuals in the same county. 

</div>

## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.